## Coursera IBM Data Science Capstone Project

By Tamela Maciel, June 2020

This jupyter notebook completes Coursera's IBM Data Science Professional Certificate capstone project.

It uses the Foursquare API, BeautifulSoup for webscraping, and various python libraries to gather location data.

In [1]:
import pandas as pd  #database wrangling
import numpy as np #linear algebra
#import string
from bs4 import BeautifulSoup #webscraping

#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

import folium # map rendering library